In [2]:
import os
import pandas as pd
import numpy as np

import re
import csv
from tqdm.auto import tqdm
import multiprocessing
import time

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService

# 샴푸&두피케어_샴푸&두피제품 찾기

In [15]:
home_url = 'https://daedamo.com/new/'
login_url = 'https://daedamo.com/new/bbs/login.php'

shamFind_url = 'https://daedamo.com/ingre?sca=%ED%83%88%EB%AA%A8%EA%B4%80%EB%A0%A8%EC%83%81%ED%92%88&bso=A&overlaps=5'
startPage = 1
endPage = 78

In [16]:
def crawReview(soup):
    commentList = soup.find_all('div', {'class', 'comment-item'})
    
    totalScoreList = []
    satisfactionScoreList = []
    priceScoreList = []
    rebuyScoreList = []
    commenterList = []
    commentDateList = []
    commentGoodList = []
    commentBadList = []
    commentContentList = []
    
    for comment in commentList:
        reviewScoreList = comment.find_all('div', {'class', 'review-item'})

        commenter = comment.find('span', {'class' : 'sv_wrap'})
        if pd.isnull(commenter):
            commenter = comment.find('span', {'class' : 'guest'}).text
        else:
            commenter = comment.find('a')['title'].replace(' 자기소개', '')
        
        commentDate = comment.find('span', {'class' : 'datetime'}).text  
        commentContent = comment.find('div', {'class' : 'contents'})
        
        
        commenterList.append(commenter)
        commentDateList.append(commentDate)
        
        
        if len(reviewScoreList) == 1:
            satisfactionScore = np.nan
            priceScore = np.nan
            rebuyScore = np.nan
            totalScore = reviewScoreList[0].find('span')['style'].replace('width:', '')[:-1]
            good = np.nan
            bad = np.nan
            commentContent = commentContent.text
                
        else:
            satisfactionScore = reviewScoreList[0].find('span')['style'].replace('width:', '')[:-1]
            priceScore = reviewScoreList[1].find('span')['style'].replace('width:', '')[:-1]
            rebuyScore = reviewScoreList[2].find('span')['style'].replace('width:', '')[:-1]
            totalScore = np.nan
            
            tmp_content = commentContent.find_all('dt')
            
            if len(tmp_content) == 2:
                commentContent = commentContent.text.split('아쉬운점')
                good = commentContent[0].split('좋은점')[1]
                bad = commentContent[1]
                
            else:
                good = commentContent.text.split('좋은점')[1]
                bad = np.nan
            commentContent = np.nan
            
        commentGoodList.append(good)
        commentBadList.append(bad)
        commentContentList.append(commentContent)      
        
        satisfactionScoreList.append(satisfactionScore)
        priceScoreList.append(priceScore)
        rebuyScoreList.append(rebuyScore)
        totalScoreList.append(totalScore)

    return [totalScoreList, satisfactionScoreList, priceScoreList, rebuyScoreList, commenterList, commentDateList, commentContentList, commentGoodList, commentBadList]

In [17]:
def saveResult(result1, resut2, savePath = '../Data/샴푸&두피제품 찾기.csv', fieldNames = ['href', 'title', 'reviewNum', 'tag', 'brand', 'company', 'howToUse', 'ingredients', 'image', 'volume', 'price', 'totalScore', 'satisfactionScore', 'priceScore', 'rebuyScore', 'commenter', 'commentDate', 'commentContent', 'commentGood', 'commentBad']):
        if result2 == []:
            with open(savePath, 'a', newline = '', encoding = 'utf-8-sig') as csvfile:
                csvWriter = csv.DictWriter(csvfile, fieldnames = fieldNames)
                csvWriter.writerow({'href' : result1[0],
                                    'title' : result1[1],
                                    'reviewNum' : result1[2],
                                    'tag' : result1[3],
                                    'brand' : result1[4],
                                    'company' : result1[5],
                                    'howToUse' : result1[6],
                                    'ingredients' : result1[7],
                                    'image' : result1[8],
                                    'volume' : result1[9],
                                    'price' : result1[10],
                                    'totalScore' : np.nan,
                                    'satisfactionScore' : np.nan,
                                    'priceScore' : np.nan,
                                    'rebuyScore' : np.nan,
                                    'commenter' : np.nan,
                                    'commentDate' : np.nan,
                                    'commentContent' : np.nan,
                                    'commentGood' : np.nan,
                                    'commentBad' : np.nan
                                   })
        else:
            tmp = [np.nan]*(len(result2[0]) -1)
            href = [result1[0]] + tmp
            title = [result1[1]] + tmp
            reviewNum = [result1[2]] + tmp
            tag = [result1[3]] + tmp
            brand = [result1[4]] + tmp
            company = [result1[5]] + tmp
            howToUse = [result1[6]] + tmp
            ingredients = [result1[7]] + tmp
            image = [result1[8]] + tmp
            volume = [result1[9]] + tmp
            price = [result1[10]] + tmp
            pd.DataFrame({'href' : href,
                          'title' : title,
                          'reviewNum' : reviewNum,
                          'tag' : tag,
                          'brand' : brand,
                          'company' : company,
                          'howToUse' : howToUse,
                          'ingredients' : ingredients,
                          'image' : image,
                          'volume' : volume,
                          'price' : price,
                          'totalScore' : result2[0],
                          'satisfactionScore' : result2[1],
                          'priceScore' : result2[2],
                          'rebuyScore' : result2[3],
                          'commenter' : result2[4],
                          'commentDate' : result2[5],
                          'commentContent' : result2[6],
                          'commentGood' : result2[7],
                          'commentBad' : result2[8],
                         }).to_csv(savePath, mode = 'a', index = False, header = None)      

In [19]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disabled-dev-shm-usage')
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
chrome_options.add_argument('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36')

savePath = '../Data/샴푸&두피제품 찾기.csv'
fieldNames = ['href', 'title', 'reviewNum', 'tag', 'brand', 'company', 'howToUse', 'ingredients', \
             'image', 'volume', 'price', 'totalScore', 'satisfactionScore', 'priceScore', \
             'rebuyScore', 'commenter', 'commentDate', 'commentContent', 'commentGood', 'commentBad']

driver = webdriver.Chrome(options = chrome_options)

driver.get(login_url)
driver.implicitly_wait(np.random.rand(1)[0])

driver.find_element_by_id('login_id').send_keys('dataflow28')
driver.find_element_by_id('login_pw').send_keys('flowdata2833!!')
driver.find_element_by_class_name('btn_submit').click()
driver.implicitly_wait(np.random.rand(1)[0])
time.sleep(2)

numCompile = re.compile('\d')

if not os.path.isfile(savePath):
    tmp = pd.DataFrame(columns = fieldNames)
    tmp.to_csv(savePath, encoding = 'utf-8-sig')
    
    with open(savePath, 'w', newline = '', encoding = 'utf-8-sig') as csvfile:
        csvWriter = csv.DictWriter(csvfile, fieldnames = fieldNames)
        csvWriter.writeheader()

for page in tqdm(range(startPage, endPage + 1)):
    url = shamFind_url + '&page=' + str(page)
    
    driver.get(url)
    driver.implicitly_wait(np.random.rand(1)[0])
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    re_list = soup.find_all('div', {'class' : 'bbs_list_box flex_spbtw'})

    for res in re_list:
        href = res.find('h3', {'class' : 'cont_subject'}).find('a')['href']
        view = res.find('div', {'class' : 'cont_info_hit'}).text

        driver.get(href)
        driver.implicitly_wait(np.random.rand(1)[0])
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        content = soup.find_all('p', {'class' : 'cont'})
        howToUse = content[0].text
        ingredients = content[1].text
        try:
            image = soup.find('img', {'class' : 'lazy'})['src']
        except:
            image = np.nan
            
        tag = soup.find('div', {'class' : 'tag'}).text
        title = soup.find('div', {'class' : 'subject'}).text
        brand = soup.find('div', {'class' : 'brand'}).text
        company = soup.find('div', {'class' : 'company'}).text
        reviewNum = int(''.join(numCompile.findall(soup.find('div', {'class' : 'score_title'}).text)))
        
        maxPageNum = 0
        if tag == '탈모의료기기':
            volume = np.nan
            price = soup.find('span', {'class' : 'price'}).text
        else:
            try:
                volumeAndPrice = soup.find('div', {'class' : 'volume_price'}).text.strip().split('\n')
                volume = volumeAndPrice[0]
                price = volumeAndPrice[1]
            except:
                volume = np.nan
                price = np.nan
        
        result1 = [href, title, reviewNum, tag, brand, company, howToUse, ingredients, image, volume, price]
        
        if reviewNum == 0:
            result2 = []
            saveResult(result1, result2)
            continue
            
        elif reviewNum <= 20:
            result2 = crawReview(soup)
            saveResult(result1, result2)
            continue
        else:
            pageNextExist = True
            nextButtonClicked = False
            
            while pageNextExist:
                pageNext = soup.find('a', {'class' : 'pg_page pg_next'})
                if pd.isnull(pageNext):
                    pageNextExist = False
                result2 = crawReview(soup)
                saveResult(result1, result2)
                
                pgPageList = driver.find_elements(By.CLASS_NAME, 'pg_page')
                pgPageList_soup = soup.find_all('a', {'class' : 'pg_page'})
                pgPageList_soup = [int(pg['data-page_no']) for pg in pgPageList_soup]
                
                for i in range(len(pgPageList)):
                    if nextButtonClicked & (i == 0):
                        continue
                    
                    if pgPageList_soup[i] > maxPageNum:
                        maxPageNum = pgPageList_soup[i]                    
                    else :
                        pageNextExist = False
                        break
                    
                    pgPageList[i].click()
                    pgPageList = driver.find_elements(By.CLASS_NAME, 'pg_page')
                    driver.implicitly_wait(np.random.rand(1)[0])
                    time.sleep(1)
                    
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    result2 = crawReview(soup)
                    saveResult(result1, result2)
                    
                    pageNext = soup.find('a', {'class' : 'pg_page pg_next'})
                    if pd.isnull(pageNext):
                        pageNextExist = False
                nextButtonClicked = True

  0%|          | 0/78 [00:00<?, ?it/s]

# 샴푸&두피케어_체험평가단

In [3]:
def clawringHref(url, pageNum, chrome_options, startPage = 1):
    if not os.path.isfile(save_path):
        tmp = pd.DataFrame(columns = fieldNames)
        tmp.to_csv(save_path, index = False, encoding = 'utf-8-sig', header=fieldNames)

    for page in tqdm(range(startPage, pageNum + 1)):
        page = str(page)
        for _ in range(3):
            try:
                response = requests.get(url + page)
                if response.status_code == 200:
                    html = response.text
                    soup = BeautifulSoup(html, 'html.parser')
                    
                    notice_list = soup.find_all('div', {'class' : ['notice_wrap']})
                    re_list = soup.find_all('div', {'class' : ['list_wrap']})
                    re_list = [str(re) for re in re_list]
                    
                    for re in re_list:
                        parsing_save(re, chrome_options)
                    break
            except requests.exceptions.ConnectionError:
                time.sleep(5)

In [4]:
def parsing_save(re, chrome_options):
    re = BeautifulSoup(re, 'html.parser')
    category = re.find('div', {'class' : 'mw_basic_list_category'}).text
    date = re.find('div', {'class' : 'mw_basic_list_datetime media-no-text'}).text
    view = re.find('div', {'class' : 'mw_basic_list_hit media-no-text'}).text
    title = re.find('h3', {'class' : 'media-list-subject'}).text.replace('\n', '')
    rank = str(re.find('img')['src']).split('/')[-1].split('.')[0]
    href = re.find_all('a')[2]['href']
    
    print(href)
    
    try:
        writer = re.find('a', {'class' : 'sv_member'}).text.strip()
    except:
        writer = re.find('div', {'mw_basic_list_name media-no-text'}).text

    content, imageList = clawringContentImage(href, chrome_options)

    try:
        pd.DataFrame({'title' : [title],
                      'category' : [category],
                      'date' : [date],
                      'writer' : [writer],
                      'rank' : [rank],
                      'view' : [view],
                      'href' : [href],
                      'content' : [content.strip()],
                      'imageList' : [imageList]}).to_csv(save_path,
                                                         mode = 'a',
                                                         index = False,
                                                         header = False,
                                                         encoding = 'utf-8-sig')
    except:
        print({'title' : title,
                'category' : category,
                'date' : date,
                'writer' : writer,
                'rank' : rank,
                'view' : view,
                'href' : href,
                'content' : content,
                'imageList' : imageList})
        print(href, ': Error', '\n')
        print('==================================================================')

In [5]:
def clawringContentImage(href, chrome_options):
    driver = webdriver.Chrome()
    driver.get(href)
    driver.implicitly_wait(np.random.rand(1)[0])
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    res = soup.find('div', {'class' : 'mw_basic_view_content'})

    try:
        content = res.text
    except:
        return np.nan, []

    images = res.find_all('img', {'class' : 'lazy content-image'})

    if images != []:
        imageList = [url['src'] for url in res.find_all('img', {'class' : 'lazy'})]
    else:
        imageList = []

    return content, imageList

In [6]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_experimental_option("detach", True) 
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disabled-dev-shm-usage')
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
chrome_options.add_argument('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36')

In [8]:
path_save_base = '../Data'
fieldNames = ['title', 'category', 'date', 'writer', 'rank', 'view', 'href', 'content', 'imageList']

path_save = '샴푸&두피케어_체험평가단.csv'
save_path = os.path.join(path_save_base, path_save)
url = 'https://daedamo.com/job?overlaps=5&page='
pageNum = 39
clawringHref(url, pageNum, chrome_options)

  0%|          | 0/39 [00:00<?, ?it/s]

https://daedamo.com/job/4616?overlaps=5
https://daedamo.com/job/4611?overlaps=5
https://daedamo.com/job/4609?overlaps=5
https://daedamo.com/job/4608?overlaps=5
https://daedamo.com/job/4606?overlaps=5
https://daedamo.com/job/4605?overlaps=5
https://daedamo.com/job/4599?overlaps=5
https://daedamo.com/job/4597?overlaps=5
https://daedamo.com/job/4596?overlaps=5
https://daedamo.com/job/4595?overlaps=5
https://daedamo.com/job/4592?overlaps=5
https://daedamo.com/job/4591?overlaps=5
https://daedamo.com/job/4590?overlaps=5
https://daedamo.com/job/4589?overlaps=5
https://daedamo.com/job/4588?overlaps=5
https://daedamo.com/job/4587?overlaps=5
https://daedamo.com/job/4586?overlaps=5
https://daedamo.com/job/4585?overlaps=5
https://daedamo.com/job/4584?overlaps=5
https://daedamo.com/job/4583?overlaps=5
https://daedamo.com/job/4582?overlaps=5
https://daedamo.com/job/4581?overlaps=5
https://daedamo.com/job/4580?overlaps=5
https://daedamo.com/job/4579?overlaps=5
https://daedamo.com/job/4578?overlaps=5


https://daedamo.com/job/4234?overlaps=5
https://daedamo.com/job/4229?overlaps=5
https://daedamo.com/job/4225?overlaps=5
https://daedamo.com/job/4224?overlaps=5
https://daedamo.com/job/4223?overlaps=5
https://daedamo.com/job/4222?overlaps=5
https://daedamo.com/job/4221?overlaps=5
https://daedamo.com/job/4220?overlaps=5
https://daedamo.com/job/4219?overlaps=5
https://daedamo.com/job/4218?overlaps=5
https://daedamo.com/job/4217?overlaps=5
https://daedamo.com/job/4216?overlaps=5
https://daedamo.com/job/4215?overlaps=5
https://daedamo.com/job/4214?overlaps=5
https://daedamo.com/job/4213?overlaps=5
https://daedamo.com/job/4212?overlaps=5
https://daedamo.com/job/4211?overlaps=5
https://daedamo.com/job/4210?overlaps=5
https://daedamo.com/job/4209?overlaps=5
https://daedamo.com/job/4208?overlaps=5
https://daedamo.com/job/4205?overlaps=5
https://daedamo.com/job/4204?overlaps=5
https://daedamo.com/job/4202?overlaps=5
https://daedamo.com/job/4201?overlaps=5
https://daedamo.com/job/4200?overlaps=5


https://daedamo.com/job/3908?overlaps=5
https://daedamo.com/job/3907?overlaps=5
https://daedamo.com/job/3906?overlaps=5
https://daedamo.com/job/3905?overlaps=5
https://daedamo.com/job/3904?overlaps=5
https://daedamo.com/job/3896?overlaps=5
https://daedamo.com/job/3895?overlaps=5
https://daedamo.com/job/3894?overlaps=5
https://daedamo.com/job/3893?overlaps=5
https://daedamo.com/job/3892?overlaps=5
https://daedamo.com/job/3889?overlaps=5
https://daedamo.com/job/3888?overlaps=5
https://daedamo.com/job/3886?overlaps=5
https://daedamo.com/job/3885?overlaps=5
https://daedamo.com/job/3884?overlaps=5
https://daedamo.com/job/3883?overlaps=5
https://daedamo.com/job/3881?overlaps=5
https://daedamo.com/job/3874?overlaps=5
https://daedamo.com/job/3873?overlaps=5
https://daedamo.com/job/3872?overlaps=5
https://daedamo.com/job/3871?overlaps=5
https://daedamo.com/job/3870?overlaps=5
https://daedamo.com/job/3868?overlaps=5
https://daedamo.com/job/3867?overlaps=5
https://daedamo.com/job/3866?overlaps=5


https://daedamo.com/job/3459?overlaps=5
https://daedamo.com/job/3458?overlaps=5
https://daedamo.com/job/3454?overlaps=5
https://daedamo.com/job/3452?overlaps=5
https://daedamo.com/job/3451?overlaps=5
https://daedamo.com/job/3450?overlaps=5
https://daedamo.com/job/3448?overlaps=5
https://daedamo.com/job/3444?overlaps=5
https://daedamo.com/job/3443?overlaps=5
https://daedamo.com/job/3442?overlaps=5
https://daedamo.com/job/3441?overlaps=5
https://daedamo.com/job/3440?overlaps=5
https://daedamo.com/job/3438?overlaps=5
https://daedamo.com/job/3433?overlaps=5
https://daedamo.com/job/3432?overlaps=5
https://daedamo.com/job/3431?overlaps=5
https://daedamo.com/job/3430?overlaps=5
https://daedamo.com/job/3429?overlaps=5
https://daedamo.com/job/3427?overlaps=5
https://daedamo.com/job/3424?overlaps=5
https://daedamo.com/job/3422?overlaps=5
https://daedamo.com/job/3418?overlaps=5
https://daedamo.com/job/3402?overlaps=5
https://daedamo.com/job/3401?overlaps=5
https://daedamo.com/job/3387?overlaps=5


https://daedamo.com/job/2930?overlaps=5
https://daedamo.com/job/2928?overlaps=5
https://daedamo.com/job/2924?overlaps=5
https://daedamo.com/job/2922?overlaps=5
https://daedamo.com/job/2916?overlaps=5
https://daedamo.com/job/2914?overlaps=5
https://daedamo.com/job/2913?overlaps=5
https://daedamo.com/job/2912?overlaps=5
https://daedamo.com/job/2910?overlaps=5
https://daedamo.com/job/2908?overlaps=5
https://daedamo.com/job/2907?overlaps=5
https://daedamo.com/job/2905?overlaps=5
https://daedamo.com/job/2900?overlaps=5
https://daedamo.com/job/2899?overlaps=5
https://daedamo.com/job/2898?overlaps=5
https://daedamo.com/job/2897?overlaps=5
https://daedamo.com/job/2896?overlaps=5
https://daedamo.com/job/2895?overlaps=5
https://daedamo.com/job/2894?overlaps=5
https://daedamo.com/job/2893?overlaps=5
https://daedamo.com/job/2892?overlaps=5
https://daedamo.com/job/2891?overlaps=5
https://daedamo.com/job/2890?overlaps=5
https://daedamo.com/job/2889?overlaps=5
https://daedamo.com/job/2888?overlaps=5


https://daedamo.com/job/2358?overlaps=5
https://daedamo.com/job/2357?overlaps=5
https://daedamo.com/job/2355?overlaps=5
https://daedamo.com/job/2354?overlaps=5
https://daedamo.com/job/2353?overlaps=5
https://daedamo.com/job/2352?overlaps=5
https://daedamo.com/job/2351?overlaps=5
https://daedamo.com/job/2350?overlaps=5
https://daedamo.com/job/2349?overlaps=5
https://daedamo.com/job/2348?overlaps=5
https://daedamo.com/job/2347?overlaps=5
https://daedamo.com/job/2346?overlaps=5
https://daedamo.com/job/2345?overlaps=5
https://daedamo.com/job/2344?overlaps=5
https://daedamo.com/job/2343?overlaps=5
https://daedamo.com/job/2342?overlaps=5
https://daedamo.com/job/2341?overlaps=5
https://daedamo.com/job/2340?overlaps=5
https://daedamo.com/job/2339?overlaps=5
https://daedamo.com/job/2338?overlaps=5
https://daedamo.com/job/2337?overlaps=5
https://daedamo.com/job/2336?overlaps=5
https://daedamo.com/job/2335?overlaps=5
https://daedamo.com/job/2334?overlaps=5
https://daedamo.com/job/2332?overlaps=5


https://daedamo.com/job/1799?overlaps=5
https://daedamo.com/job/1798?overlaps=5
https://daedamo.com/job/1797?overlaps=5
https://daedamo.com/job/1796?overlaps=5
https://daedamo.com/job/1794?overlaps=5
https://daedamo.com/job/1793?overlaps=5
https://daedamo.com/job/1792?overlaps=5
https://daedamo.com/job/1791?overlaps=5
https://daedamo.com/job/1790?overlaps=5
https://daedamo.com/job/1789?overlaps=5
https://daedamo.com/job/1788?overlaps=5
https://daedamo.com/job/1787?overlaps=5
https://daedamo.com/job/1786?overlaps=5
https://daedamo.com/job/1785?overlaps=5
https://daedamo.com/job/1784?overlaps=5
https://daedamo.com/job/1783?overlaps=5
https://daedamo.com/job/1782?overlaps=5
https://daedamo.com/job/1781?overlaps=5
https://daedamo.com/job/1780?overlaps=5
https://daedamo.com/job/1754?overlaps=5
https://daedamo.com/job/1753?overlaps=5
https://daedamo.com/job/1677?overlaps=5
https://daedamo.com/job/1355?overlaps=5
https://daedamo.com/job/1240?overlaps=5
{'title': '체험단신청합니다', 'category': ' [DHT

https://daedamo.com/job/501?overlaps=5
https://daedamo.com/job/500?overlaps=5
https://daedamo.com/job/499?overlaps=5
https://daedamo.com/job/498?overlaps=5
https://daedamo.com/job/497?overlaps=5
https://daedamo.com/job/496?overlaps=5
https://daedamo.com/job/494?overlaps=5
https://daedamo.com/job/493?overlaps=5
https://daedamo.com/job/482?overlaps=5
https://daedamo.com/job/481?overlaps=5
https://daedamo.com/job/477?overlaps=5
https://daedamo.com/job/474?overlaps=5
https://daedamo.com/job/473?overlaps=5
https://daedamo.com/job/472?overlaps=5
https://daedamo.com/job/471?overlaps=5
https://daedamo.com/job/468?overlaps=5
https://daedamo.com/job/467?overlaps=5
https://daedamo.com/job/465?overlaps=5
https://daedamo.com/job/464?overlaps=5
https://daedamo.com/job/463?overlaps=5
https://daedamo.com/job/452?overlaps=5
https://daedamo.com/job/449?overlaps=5
https://daedamo.com/job/445?overlaps=5
https://daedamo.com/job/430?overlaps=5
https://daedamo.com/job/427?overlaps=5
https://daedamo.com/job/4

WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=115.0.5790.170)


In [ ]:
!sudo poweroff